In [1]:
import torch
from sklearn.datasets import fetch_openml

from neural_blueprints.architectures import MLP
from neural_blueprints.config.architectures import MLPConfig
from neural_blueprints.config.utils import TrainerConfig
from neural_blueprints.config.components.composite.projections import TabularProjectionConfig
from neural_blueprints.utils import Trainer, infer_types
from neural_blueprints.preprocess import TabularPreprocessor
from neural_blueprints.datasets import MaskedTabularDataset, TabularLabelDataset

import logging
logging.basicConfig(
    level=logging.DEBUG,  # or DEBUG if you want even more detail
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

In [2]:
data = fetch_openml(name="adult", version=2, as_frame=True)
X = data.data
y = data.target

data = X.copy()
data['income'] = y

dtypes = infer_types(data)
data = data.astype(dtypes)
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
preprocessor = TabularPreprocessor()
data, discrete_features, continuous_features = preprocessor.run(data)
data.head()

2026-01-22 16:57:07,860 - neural_blueprints.preprocess.tabular_preprocess - INFO - Identified 10 discrete features: ['workclass', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'income']
2026-01-22 16:57:07,861 - neural_blueprints.preprocess.tabular_preprocess - INFO - Identified 5 continuous features: ['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']
2026-01-22 16:57:07,862 - neural_blueprints.preprocess.tabular_preprocess - INFO - Discrete column 'workclass' has 2799/5.73% NaN values; these will be encoded as 0.
2026-01-22 16:57:07,926 - neural_blueprints.preprocess.tabular_preprocess - INFO - Discrete column 'occupation' has 2809/5.75% NaN values; these will be encoded as 0.
2026-01-22 16:57:07,989 - neural_blueprints.preprocess.tabular_preprocess - INFO - Discrete column 'native-country' has 857/1.75% NaN values; these will be encoded as 0.


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.109589,4,0.145129,2,14,5,7,4,3,2,0.000000,0.0,0.397959,39,1
1,0.287671,4,0.052451,12,16,3,5,1,5,2,0.000000,0.0,0.500000,39,1
2,0.150685,2,0.219649,8,4,3,11,1,5,2,0.000000,0.0,0.397959,39,2
3,0.369863,4,0.100153,16,2,3,7,1,3,2,0.076881,0.0,0.397959,39,2
4,0.013699,0,0.061708,16,2,5,0,4,5,1,0.000000,0.0,0.295918,39,1


### Income Inference Accuracy

In [4]:
dataset = TabularLabelDataset(
    data=data,
    label_columns=['income'],              # Specify the label column for single-label classification
)

train_dataset, val_dataset = dataset.random_split([0.9, 0.1])

In [7]:
# Define model configuration
latent_dim = 32

mlp_config = MLPConfig(
    input_spec=(len(dataset.cardinalities),),
    output_spec=(3,),
    input_projection=TabularProjectionConfig(
        input_cardinalities=dataset.cardinalities,
        hidden_dims=[latent_dim*2, latent_dim*2],
        projection_dim=latent_dim,
        output_dim=[len(dataset.cardinalities)*latent_dim],
    ),
    hidden_dims=[256, 128, 128, 64, 64],
    normalization="batchnorm1d",
    activation='gelu',
    dropout_p=0.1,
    final_activation="sigmoid",
)

# Initialize model
model = MLP(mlp_config)
model.blueprint(batch_size=256, with_graph=False)

2026-01-22 16:58:10,877 - neural_blueprints.architectures.mlp - INFO - Using input projection: TabularInputProjection


Layer (type:depth-idx)                                                           Output Shape              Param #
MLP                                                                              [256, 3]                  --
├─Sequential: 1-3                                                                --                        (recursive)
│    └─TabularInputProjection: 2-1                                               [256, 448]                --
│    │    └─ModuleList: 3-1                                                      --                        138,304
├─Sequential: 1-2                                                                [256, 3]                  178,243
├─Sequential: 1-3                                                                --                        (recursive)
│    └─Sequential: 2-2                                                           --                        --
│    │    └─FeedForwardNetwork: 3-2                                              [256, 

In [8]:
trainer = Trainer(
    config=TrainerConfig(
        optimizer="adam",
        criterion="cross_entropy",
        learning_rate=3e-4,
        weight_decay=1e-4,
        batch_size=256,
        early_stopping_patience=5
    ),
    model= model
)

# Train the model
trainer.train(train_dataset, val_dataset, epochs=5)
trainer.predict(val_dataset)

2026-01-22 16:58:12,149 - neural_blueprints.utils.trainer - INFO - Trainer initialized on device: cpu
Training Epochs:  20%|██        | 1/5 [00:02<00:10,  2.70s/epoch]

Epoch 1/5, Training Loss: 0.8945, Validation Loss: 0.7815


Training Epochs:  40%|████      | 2/5 [00:05<00:07,  2.62s/epoch]

Epoch 2/5, Training Loss: 0.7532, Validation Loss: 0.7197


Training Epochs:  60%|██████    | 3/5 [00:07<00:05,  2.59s/epoch]

...

Training Epochs: 100%|██████████| 5/5 [00:12<00:00,  2.59s/epoch]
2026-01-22 16:58:25,118 - neural_blueprints.utils.trainer - INFO - Training completed in 12.97 seconds.
2026-01-22 16:58:25,118 - neural_blueprints.utils.trainer - INFO - Best validation loss: 6.8855e-01


Epoch 5/5, Training Loss: 0.6864, Validation Loss: 0.6886


2026-01-22 16:58:25,228 - neural_blueprints.utils.trainer - INFO - Inference completed in 0.02 seconds.


Classification Accuracy: 0.8465


0.8464687819856704

### Masked Dataset Inference Accuracy

In [10]:
dataset = MaskedTabularDataset(
    data = data,
    mask_prob=0.15
)

train_dataset, val_dataset, test_dataset = dataset.random_split([0.9, 0.05, 0.05])

In [15]:
# Define model configuration
latent_dim = 32

mlp_config = MLPConfig(
    input_spec=(len(dataset.cardinalities),),
    output_spec=(len(dataset.cardinalities),),
    input_projection=TabularProjectionConfig(
        input_cardinalities=dataset.cardinalities,
        projection_dim=latent_dim,
        output_dim=[len(dataset.cardinalities) * latent_dim],
    ),
    output_projection=TabularProjectionConfig(
        input_dim=[latent_dim],
        hidden_dims=[64, 64],
        output_cardinalities=dataset.cardinalities,
    ),
    hidden_dims=[512, 256, 128, 64, latent_dim],
    normalization="batchnorm1d",
    activation="gelu",
    dropout_p=0.1,
    final_activation=None,
)

# Initialize model
model = MLP(mlp_config)
model.blueprint(batch_size=256, with_graph=False)

2026-01-22 17:06:38,324 - neural_blueprints.architectures.mlp - INFO - Using input projection: TabularInputProjection
2026-01-22 17:06:38,335 - neural_blueprints.architectures.mlp - INFO - Using output projection: TabularOutputProjection


Layer (type:depth-idx)                                                           Output Shape              Param #
MLP                                                                              [256, 1]                  --
├─Sequential: 1-3                                                                --                        (recursive)
│    └─TabularInputProjection: 2-1                                               [256, 480]                --
│    │    └─ModuleList: 3-1                                                      --                        5,120
├─Sequential: 1-2                                                                [256, 32]                 --
│    └─FeedForwardNetwork: 2-2                                                   [256, 32]                 --
│    │    └─Sequential: 3-2                                                      [256, 32]                 423,872
├─Sequential: 1-3                                                                --               

In [16]:
trainer = Trainer(
    config=TrainerConfig(
        optimizer="adam",
        criterion="masked_reconstruction",
        learning_rate=3e-4,
        weight_decay=1e-4,
        batch_size=256,
        early_stopping_patience=5
    ),
    model= model
)

# Train the model
trainer.train(train_dataset, val_dataset, epochs=5)
trainer.predict(test_dataset)

2026-01-22 17:06:41,392 - neural_blueprints.utils.trainer - INFO - Trainer initialized on device: cpu
Training Epochs:  20%|██        | 1/5 [00:03<00:12,  3.04s/epoch]

Epoch 1/5, Training Loss: 3.0626, Validation Loss: 2.7475


Training Epochs:  40%|████      | 2/5 [00:05<00:08,  2.98s/epoch]

Epoch 2/5, Training Loss: 2.7230, Validation Loss: 2.6068


Training Epochs:  60%|██████    | 3/5 [00:08<00:05,  2.94s/epoch]

...

Training Epochs: 100%|██████████| 5/5 [00:14<00:00,  2.94s/epoch]
2026-01-22 17:06:56,121 - neural_blueprints.utils.trainer - INFO - Training completed in 14.73 seconds.
2026-01-22 17:06:56,121 - neural_blueprints.utils.trainer - INFO - Best validation loss: 2.5367e+00


Epoch 5/5, Training Loss: 2.5861, Validation Loss: 2.5367


2026-01-22 17:06:56,205 - neural_blueprints.utils.predict - WARNING - Warning: Only one class predicted for feature column 1.
2026-01-22 17:06:56,208 - neural_blueprints.utils.predict - WARNING - Warning: Only one class predicted for feature column 13.
2026-01-22 17:06:56,208 - neural_blueprints.utils.trainer - INFO - Inference completed in 0.03 seconds.


Feature Column 0:
Predicted attribute values: [0.42252967 0.3443459  0.3968442  0.31401438 0.29175597]
True attribute values: [0.41095892 0.24657534 0.10958904 0.9726027  0.6712329 ]
Accuracy: 0.1890

Feature Column 1:
Predicted attribute values: [4 4 4 4 4]
True attribute values: [6. 6. 5. 4. 4.]
Accuracy: 0.6931

Feature Column 2:
Predicted attribute values: [0.1241991  0.14244592 0.14818011 0.13715005 0.11498227]
True attribute values: [0.20159121 0.10526718 0.08111818 0.07875842 0.16413134]
Accuracy: 0.5414

Feature Column 3:
Predicted attribute values: [12 10 12 10 10]
True attribute values: [10. 10. 16. 10. 10.]
Accuracy: 0.6576

Feature Column 4:
Predicted attribute values: [ 2 16 16  5  5]
True attribute values: [2. 3. 5. 5. 5.]
Accuracy: 0.5795

Feature Column 5:
Predicted attribute values: [3 3 5 5 5]
True attribute values: [3. 3. 3. 5. 5.]
Accuracy: 0.7413

Feature Column 6:
Predicted attribute values: [ 4 10  8  0  0]
True attribute values: [12.  1.  8.  6.  1.]
Accuracy: 0

{'avg_discrete_accuracy': np.float64(0.7011779202007863),
 'avg_continuous_accuracy': np.float64(0.5447715434860857),
 'overall_avg_accuracy': np.float64(0.649042461295886)}